In [1]:
import regex as re
import pandas as pd
import numpy as np

/tmp/ipykernel_9765/314147691.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Loading Base Code

In [16]:
def readCode(file_name):
    base_code = ""
    with open (file_name,'r') as f:
        for line in f:
            base_code+=line
    return base_code

### Loading and Cleaning pixel code

In [15]:
def extractConfigurationCode(file_name):
    pixel_code = ""

    with open (file_name,'r') as f:
        for line in f:
            pixel_code+=line
    configuration = "fbq.registerPlugin" + (pixel_code.split('fbq.registerPlugin')[1].split('/*')[0])
    return configuration

In [24]:
import difflib

code1 = extractConfigurationCode('temp_files/temp5.html')
code2 = extractConfigurationCode('temp_files/temp6.html')

# Split the code into lines
code1_lines = code1.strip().splitlines()
code2_lines = code2.strip().splitlines()

# Create an instance of HtmlDiff
html_diff = difflib.HtmlDiff(wrapcolumn=100)

# Generate the HTML table with customized styling
html_result = html_diff.make_file(
    code1_lines, 
    code2_lines, 
    context=True,  # Show only the differences with context
    numlines=2     # Number of context lines to show
)

# Custom HTML styling to enhance visual appeal
custom_css = """
<style>
    body {
        font-family: Arial, sans-serif;
        line-height: 1.6;
        background-color: #f4f4f4;
        margin: 0;
        padding: 20px;
    }
    table.diff {
        width: 100%;
        border-collapse: collapse;
        margin: 20px 0;
        font-size: 16px;
    }
    table.diff th {
        background-color: #2c3e50;
        color: white;
        padding: 10px;
        text-align: left;
    }
    table.diff td {
        padding: 10px;
        vertical-align: top;
        border-bottom: 1px solid #ddd;
    }
    .diff_header {
        background-color: #34495e;
        color: white;
    }
    .diff_next {
        background-color: #f39c12;
    }
    .diff_add {
        background-color: #2ecc71;
        color: white;
    }
    .diff_sub {
        background-color: #e74c3c;
        color: white;
    }
    .diff_chg {
        background-color: #3498db;
        color: white;
    }
</style>
"""

# Embed the custom CSS into the HTML result
html_result = html_result.replace(
    '<style type="text/css">', 
    '<style type="text/css">' + custom_css
)

# Save the result to an HTML file
with open('diff_successive.html', 'w') as file:
    file.write(html_result)

print("Differences saved to diff_successive.html")

# Collect added and deleted parts
diff = difflib.ndiff(code1_lines, code2_lines)
added_parts = []
deleted_parts = []

for line in diff:
    if line.startswith('+ '):
        added_parts.append(line[2:])
    elif line.startswith('- '):
        deleted_parts.append(line[2:])

# Print added and deleted parts
print("Added parts:")
print("\n".join(added_parts))

print("\nDeleted parts:")
print("\n".join(deleted_parts))


Differences saved to diff_successive.html
Added parts:
instance.optIn("1264059003707256", "AutomaticMatchingForPartnerIntegrations", true);instance.configLoaded("1264059003707256"); }});

Deleted parts:
instance.optIn("1264059003707256", "AutomaticMatchingForPartnerIntegrations", true);
config.set(null, "batching", {"batchWaitTimeMs":501,"maxBatchSize":10});
config.set(null, "microdata", {"waitTimeMs":500});
config.set("1264059003707256", "prohibitedSources", {"prohibitedSources":[{"domain":"41ddbaef2bda3557f4cd1c64698071ceb398dd3efbd022c3b06ab5622d22b014"},{"domain":"1b895868bf7cb1d2a9c94f7c7fd3309af42c3f79f3ab6c29738933e7a0fdb32f"},{"domain":"0e88b94c8911bee47cf8b9a3fb14719b1cbac664fce7e754e54520f0250251c0"},{"domain":"6eba6edf984bf767ab88553aed11872a8842cc576daee425e12fb16f6722ad06"},{"domain":"66c929d0906871d680ffe0b914f1ff7cf407d73f355a5042f23818c440342031"},{"domain":"b5758efaa0de66e3065f5b8db41230035b312ceeb6ef15f0e1090c05beeb38cd"},{"domain":"b7db5ac8a0aa3fc0a915776b6426880451b

## Analyzing Changes in Time (i.e. parts added and parts deleted)

In [21]:
import pandas as pd
import difflib
import os

# Directory containing the HTML files
folder_path = 'riteaid_archived_versions'

# Get all files in the directory sorted by their timestamps
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith('.html')])

# DataFrame to store the results
columns = ['Timestamp1', 'Timestamp2', 'Code1', 'Code2', 'Added Parts', 'Deleted Parts']
df = pd.DataFrame(columns=columns)

# List to accumulate rows before creating DataFrame
rows = []

# Iterate over the files successively
for i in range(len(file_list) - 1):
    file1 = file_list[i]
    file2 = file_list[i + 1]

    # Get timestamps
    timestamp1 = file1.split('.')[0]
    timestamp2 = file2.split('.')[0]

    # Extract code
    code1 = extractConfigurationCode(os.path.join(folder_path, file1))
    code2 = extractConfigurationCode(os.path.join(folder_path, file2))

    # Split the code into lines
    code1_lines = code1.strip().splitlines()
    code2_lines = code2.strip().splitlines()

    # Create an instance of HtmlDiff
    html_diff = difflib.HtmlDiff(wrapcolumn=100)

    # Generate the HTML table with customized styling
    html_result = html_diff.make_file(
        code1_lines, 
        code2_lines, 
        context=True,  # Show only the differences with context
        numlines=2     # Number of context lines to show
    )

    # Collect added and deleted parts
    diff = difflib.ndiff(code1_lines, code2_lines)
    added_parts = []
    deleted_parts = []

    for line in diff:
        if line.startswith('+ '):
            added_parts.append(line[2:])
        elif line.startswith('- '):
            deleted_parts.append(line[2:])

    # Store results in rows list
    rows.append({
        'Timestamp1': timestamp1,
        'Timestamp2': timestamp2,
        'Code1': code1,
        'Code2': code2,
        'Added Parts': '\n'.join(added_parts),
        'Deleted Parts': '\n'.join(deleted_parts)
    })

# Convert list of rows to DataFrame
df = pd.DataFrame(rows, columns=columns)

# Save the DataFrame to a CSV file
df.to_csv('code_comparisons.csv', index=False)

print("Comparison results saved to code_comparisons.csv")


Comparison results saved to code_comparisons.csv


In [23]:
zamn = pd.read_csv('code_comparisons.csv')